In [11]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

PATH = Path("pg/")
list(PATH.iterdir())

[PosixPath('pg/route_ratings.csv'),
 PosixPath('pg/routes.csv'),
 PosixPath('pg/users.csv')]

In [12]:
ratings = pd.read_csv(PATH/"route_ratings.csv")
users = pd.read_csv(PATH/"users.csv")
routes = pd.read_csv(PATH/"routes.csv")
ratings.head()

,id,route_id,user_id,rating,comment
0,1,65,4,3,NaN
1,2,63,4,2,NaN
2,3,53,4,5,NaN
3,4,65,13,1,NaN
4,5,148,13,3,NaN


In [13]:
Mean = ratings.groupby(by="user_id",as_index=False)['rating'].mean()
Rating_avg = pd.merge(ratings,Mean,on='user_id')
Rating_avg.head()

,id,route_id,user_id,rating_x,comment,rating_y
0,1,65,4,3,NaN,3.333333
1,2,63,4,2,NaN,3.333333
2,3,53,4,5,NaN,3.333333
3,4,65,13,1,NaN,3.555556
4,5,148,13,3,NaN,3.555556


In [14]:
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,id,route_id,user_id,rating_x,comment,rating_y,adg_rating
0,1,65,4,3,NaN,3.333333,-0.333333
1,2,63,4,2,NaN,3.333333,-1.333333
2,3,53,4,5,NaN,3.333333,1.666667
3,4,65,13,1,NaN,3.555556,-2.555556
4,5,148,13,3,NaN,3.555556,-0.555556


In [15]:
check = pd.pivot_table(Rating_avg,values='rating_x',index='user_id',columns='route_id')
check.head()

route_id,53,63,64,65,72,73,148,150,152,153,165,166,171,173,174,175,176,178,212,213
user_id,,,,,,,,,,,,,,,,,,,,
4,5.0,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,4.0,NaN,NaN,1.0,NaN,NaN,3.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,NaN,1.666667,5.0,5.0,2.0,4.0,3.0,NaN,NaN
57,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,5.0,5.0,2.75,3.5,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [16]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='user_id',columns='route_id')
final.head()

route_id,53,63,64,65,72,73,148,150,152,153,165,166,171,173,174,175,176,178,212,213
user_id,,,,,,,,,,,,,,,,,,,,
4,1.666667,-1.333333,NaN,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,0.444444,NaN,NaN,-2.555556,NaN,NaN,-0.555556,NaN,1.444444,1.444444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,-1.333333,2.0,2.0,-1.0,1.0,0.0,NaN,NaN
57,NaN,NaN,NaN,-1.562500,-1.5625,-1.5625,NaN,-1.5625,2.437500,2.437500,0.1875,0.9375,NaN,NaN,NaN,NaN,NaN,NaN,-1.5625,-1.5625


In [17]:
# Replacing NaN by Route Average
final_route = final.fillna(final.mean(axis=0))
final_route.head()

route_id,53,63,64,65,72,73,148,150,152,153,165,166,171,173,174,175,176,178,212,213
user_id,,,,,,,,,,,,,,,,,,,,
4,1.666667,-1.333333,-1.0,-0.333333,-1.5625,-1.5625,-0.555556,-1.5625,1.940972,1.940972,0.0625,0.96875,-1.333333,2.0,2.0,-1.0,1.0,0.0,-1.5625,-1.5625
13,0.444444,-1.333333,-1.0,-2.555556,-1.5625,-1.5625,-0.555556,-1.5625,1.444444,1.444444,0.0625,0.96875,-1.333333,2.0,2.0,-1.0,1.0,0.0,-1.5625,-1.5625
38,1.055556,-1.333333,-1.0,-1.483796,-1.5625,-1.5625,-0.555556,-1.5625,1.940972,1.940972,0.0000,1.00000,-1.333333,2.0,2.0,-1.0,1.0,0.0,-1.5625,-1.5625
56,1.055556,-1.333333,-1.0,-1.483796,-1.5625,-1.5625,-0.555556,-1.5625,1.940972,1.940972,0.0000,0.96875,-1.333333,2.0,2.0,-1.0,1.0,0.0,-1.5625,-1.5625
57,1.055556,-1.333333,-1.0,-1.562500,-1.5625,-1.5625,-0.555556,-1.5625,2.437500,2.437500,0.1875,0.93750,-1.333333,2.0,2.0,-1.0,1.0,0.0,-1.5625,-1.5625


In [18]:
# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)
final_user.head()

route_id,53,63,64,65,72,73,148,150,152,153,165,166,171,173,174,175,176,178,212,213
user_id,,,,,,,,,,,,,,,,,,,,
4,1.666667,-1.333333,-1.480297e-16,-0.333333,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16,-1.480297e-16
13,0.444444,0.044444,4.444444e-02,-2.555556,4.444444e-02,4.444444e-02,-5.555556e-01,4.444444e-02,1.444444e+00,1.444444e+00,4.444444e-02,4.444444e-02,4.444444e-02,4.444444e-02,4.444444e-02,4.444444e-02,4.444444e-02,4.444444e-02,4.444444e-02,4.444444e-02
38,0.000000,0.000000,-1.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
56,0.380952,0.380952,3.809524e-01,0.380952,3.809524e-01,3.809524e-01,3.809524e-01,3.809524e-01,3.809524e-01,3.809524e-01,0.000000e+00,3.809524e-01,-1.333333e+00,2.000000e+00,2.000000e+00,-1.000000e+00,1.000000e+00,0.000000e+00,3.809524e-01,3.809524e-01
57,-0.337500,-0.337500,-3.375000e-01,-1.562500,-1.562500e+00,-1.562500e+00,-3.375000e-01,-1.562500e+00,2.437500e+00,2.437500e+00,1.875000e-01,9.375000e-01,-3.375000e-01,-3.375000e-01,-3.375000e-01,-3.375000e-01,-3.375000e-01,-3.375000e-01,-1.562500e+00,-1.562500e+00


In [19]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

user_id,4,13,38,56,57
user_id,,,,,
4,0.000000e+00,0.211719,0.000000,-2.359224e-16,0.035355
13,2.117189e-01,0.000000,0.000000,2.732432e-02,0.594450
38,0.000000e+00,0.000000,0.000000,0.000000e+00,0.168628
56,-2.359224e-16,0.027324,0.000000,0.000000e+00,-0.140231
57,3.535469e-02,0.594450,0.168628,-1.402310e-01,0.000000


In [20]:
# user similarity on replacing NAN by item(route) avg
cosine = cosine_similarity(final_route)
np.fill_diagonal(cosine, 0 )
similarity_with_route = pd.DataFrame(cosine,index=final_route.index)
similarity_with_route.columns=final_user.index
similarity_with_route.head()

user_id,4,13,38,56,57
user_id,,,,,
4,0.000000,0.910489,0.978019,0.978010,0.972561
13,0.910489,0.000000,0.974047,0.974039,0.960774
38,0.978019,0.974047,0.000000,0.999988,0.994950
56,0.978010,0.974039,0.999988,0.000000,0.995025
57,0.972561,0.960774,0.994950,0.995025,0.000000


In [21]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [22]:
# top 3 neighbours for each user
sim_user_3_u = find_n_neighbours(similarity_with_user,3)
sim_user_3_u.head()

,top1,top2,top3
user_id,,,
4,13,57,38
13,57,4,56
38,57,56,38
56,13,56,38
57,13,38,4


In [23]:
sim_user_3_m = find_n_neighbours(similarity_with_route,3)
sim_user_3_m.head()

,top1,top2,top3
user_id,,,
4,38,56,57
13,38,56,57
38,56,57,4
56,38,57,4
57,56,38,4


In [27]:
def get_user_similar_routes( user1, user2 ):
    common_routes = Rating_avg[Rating_avg.user_id == user1].merge(
    Rating_avg[Rating_avg.user_id == user2],
    on = "route_id",
    how = "inner" )
    return common_routes.merge( routes, on = 'route_id' )

In [28]:
a = get_user_similar_routes(4,13)
a = a.loc[ : , ['rating_x_x','rating_x_y','route_id']]
a.head()

NameError: name 'routes' is not defined